In [1]:
class KNNClassifier:
    def __init__(self, k=3):
        self.k = k

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def predict(self, X_test):
        return [self._predict(x) for x in X_test]

    def _predict(self, x):
        distances = [(self._euclidean_distance(x, x_train), label)
                     for x_train, label in zip(self.X_train, self.y_train)]
        distances.sort()
        k_nearest = [label for _, label in distances[:self.k]]
        return max(set(k_nearest), key=k_nearest.count)

    def _euclidean_distance(self, x1, x2):
        return sum((a - b) ** 2 for a, b in zip(x1, x2)) ** 0.5

In [2]:
def accuracy(y_true, y_pred):
    return sum(yt == yp for yt, yp in zip(y_true, y_pred)) / len(y_true)

def confusion_matrix(y_true, y_pred, labels):
    matrix = [[0 for _ in labels] for _ in labels]
    label_to_index = {label: idx for idx, label in enumerate(labels)}
    for yt, yp in zip(y_true, y_pred):
        matrix[label_to_index[yt]][label_to_index[yp]] += 1
    return matrix

def precision_recall_f1(y_true, y_pred):
    labels = list(set(y_true))
    scores = {}
    for label in labels:
        tp = sum((yt == label and yp == label) for yt, yp in zip(y_true, y_pred))
        fp = sum((yt != label and yp == label) for yt, yp in zip(y_true, y_pred))
        fn = sum((yt == label and yp != label) for yt, yp in zip(y_true, y_pred))

        prec = tp / (tp + fp) if (tp + fp) else 0
        rec = tp / (tp + fn) if (tp + fn) else 0
        f1 = 2 * prec * rec / (prec + rec) if (prec + rec) else 0

        scores[label] = {'precision': prec, 'recall': rec, 'f1': f1}
    return scores

In [5]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

iris = load_iris()
X, y = iris.data.tolist(), iris.target.tolist()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

model = KNNClassifier(k=3)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("Accuracy:", accuracy(y_test, y_pred))
print("Confusion Matrix:", confusion_matrix(y_test, y_pred, list(set(y))))
print("Precision, Recall, F1:", precision_recall_f1(y_test, y_pred))

Accuracy: 0.9555555555555556
Confusion Matrix: [[17, 0, 0], [0, 16, 2], [0, 0, 10]]
Precision, Recall, F1: {0: {'precision': 1.0, 'recall': 1.0, 'f1': 1.0}, 1: {'precision': 1.0, 'recall': 0.8888888888888888, 'f1': 0.9411764705882353}, 2: {'precision': 0.8333333333333334, 'recall': 1.0, 'f1': 0.9090909090909091}}


In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load Kaggle news dataset
df = pd.read_json('News_Category_Dataset_v3.json', lines=True)
df = df[['headline', 'category']]
df = df[df['category'].isin(['POLITICS', 'SPORTS', 'TECH'])]  # Keep 3 categories for simplicity
texts = df['headline'].tolist()
labels = df['category'].tolist()

df.shape

(42783, 2)

In [7]:
for k in range(1, 11):
    for test_size in [0.2, 0.3, 0.4]:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
        model = KNNClassifier(k=k)
        model.fit(X_train, y_train)
        preds = model.predict(X_test)
        acc = accuracy(y_test, preds)
        print(f"k={k}, split={test_size}, accuracy={acc:.2f}")

k=1, split=0.2, accuracy=0.93
k=1, split=0.3, accuracy=0.96
k=1, split=0.4, accuracy=0.98
k=2, split=0.2, accuracy=0.97
k=2, split=0.3, accuracy=0.96
k=2, split=0.4, accuracy=0.95
k=3, split=0.2, accuracy=1.00
k=3, split=0.3, accuracy=0.91
k=3, split=0.4, accuracy=0.95
k=4, split=0.2, accuracy=0.97
k=4, split=0.3, accuracy=0.98
k=4, split=0.4, accuracy=0.98
k=5, split=0.2, accuracy=1.00
k=5, split=0.3, accuracy=0.96
k=5, split=0.4, accuracy=0.97
k=6, split=0.2, accuracy=0.97
k=6, split=0.3, accuracy=1.00
k=6, split=0.4, accuracy=0.97
k=7, split=0.2, accuracy=0.93
k=7, split=0.3, accuracy=0.96
k=7, split=0.4, accuracy=0.93
k=8, split=0.2, accuracy=0.90
k=8, split=0.3, accuracy=0.98
k=8, split=0.4, accuracy=0.95
k=9, split=0.2, accuracy=0.97
k=9, split=0.3, accuracy=0.96
k=9, split=0.4, accuracy=0.95
k=10, split=0.2, accuracy=1.00
k=10, split=0.3, accuracy=0.96
k=10, split=0.4, accuracy=0.92


In [8]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=3)
model.fit(X_train, y_train)
sk_preds = model.predict(X_test)

print("Sklearn Accuracy:", accuracy(y_test, sk_preds))
print("Sklearn F1:", precision_recall_f1(y_test, sk_preds))

Sklearn Accuracy: 0.9
Sklearn F1: {0: {'precision': np.float64(1.0), 'recall': np.float64(1.0), 'f1': np.float64(1.0)}, 1: {'precision': np.float64(0.8181818181818182), 'recall': np.float64(0.9), 'f1': np.float64(0.8571428571428572)}, 2: {'precision': np.float64(0.9), 'recall': np.float64(0.8181818181818182), 'f1': np.float64(0.8571428571428572)}}


In [9]:

import pandas as pd
from sklearn.model_selection import train_test_split

# Load Kaggle news dataset
df = pd.read_json('News_Category_Dataset_v3.json', lines=True)
df = df[['headline', 'category']]
df = df[df['category'].isin(['POLITICS', 'SPORTS', 'TECH'])]  # Keep 3 categories for simplicity
texts = df['headline'].tolist()
labels = df['category'].tolist()

# Use 50% of the dataset, split as 40% training and 10% testing
from sklearn.utils import shuffle
texts, labels = shuffle(texts, labels, random_state=42)
limit = int(0.1 * len(texts))
texts = texts[:limit]
labels = labels[:limit]

# TF-IDF preprocessing

def tokenize(text):
    return text.lower().split()

def compute_tf(text):
    tokens = tokenize(text)
    tf = {}
    for word in tokens:
        tf[word] = tf.get(word, 0) + 1
    total = len(tokens)
    for word in tf:
        tf[word] /= total
    return tf

def compute_idf(corpus):
    import math
    idf = {}
    total_docs = len(corpus)
    for doc in corpus:
        for word in set(tokenize(doc)):
            idf[word] = idf.get(word, 0) + 1
    for word in idf:
        idf[word] = math.log(total_docs / idf[word])
    return idf

def compute_tfidf(text, idf):
    tf = compute_tf(text)
    return {word: tf[word] * idf.get(word, 0) for word in tf}

def vectorize(tfidf_dict, vocab):
    return [tfidf_dict.get(word, 0.0) for word in vocab]

idf = compute_idf(texts)
vocab = sorted(idf.keys())
X = [vectorize(compute_tfidf(text, idf), vocab) for text in texts]
y = labels

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.02)  # 40% train, 10% test from 50% total

model = KNNClassifier(k=3)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("News Accuracy:", accuracy(y_test, y_pred))
print("News F1:", precision_recall_f1(y_test, y_pred))

News Accuracy: 0.7790697674418605
News F1: {'TECH': {'precision': 0.5, 'recall': 0.25, 'f1': 0.3333333333333333}, 'SPORTS': {'precision': 1.0, 'recall': 0.058823529411764705, 'f1': 0.1111111111111111}, 'POLITICS': {'precision': 0.7831325301204819, 'recall': 1.0, 'f1': 0.8783783783783784}}
